In [35]:
import pandas as pd

# Path to your CSV file
file_path = "/Users/thomasclaiborne/Downloads/training_data.csv"

chunk_size = 100000  # Number of rows per chunk
chunks = pd.read_csv(file_path, chunksize=chunk_size)



In [36]:
# Combine all chunks into a single DataFrame
df = pd.concat(chunks)


/var/folders/4r/yb6kq92n7nv16sy9y4wfcjww0000gp/T/ipykernel_33676/1543202016.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunks)
/var/folders/4r/yb6kq92n7nv16sy9y4wfcjww0000gp/T/ipykernel_33676/1543202016.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunks)
/var/folders/4r/yb6kq92n7nv16sy9y4wfcjww0000gp/T/ipykernel_33676/1543202016.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunks)
/var/folders/4r/yb6kq92n7nv16sy9y4wfcjww0000gp/T/ipykernel_33676/1543202016.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunks)
/var/folders/4r/yb6kq92n7nv16sy9y4wfcjww0000gp/T/ipykernel_33676/1543202016.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dty

In [37]:
# Define required columns
required_columns = ['TaggedPitchType', 'AutoPitchType', 'PitchCall', 'KorBB', 'TaggedHitType',
                    'PlayResult', 'RunsScored', 'RelSpeed', 'VertRelAngle', 'HorzRelAngle',
                    'SpinRate', 'SpinAxis', 'Tilt', 'RelHeight', 'RelSide', 'Extension','InducedVertBreak',
                    'HorzBreak', 'VertApprAngle', 'HorzApprAngle','PlateLocHeight','PlateLocSide',
                    'ExitSpeed']

# Create new dataframe with only required columns
df_relevant = df[required_columns]

# Print info about the new dataframe
print(f"Original DataFrame shape: {df.shape}")
print(f"New DataFrame shape: {df_relevant.shape}")
print("\nNew DataFrame info:")
print(df_relevant.info())


Original DataFrame shape: (1068473, 167)
New DataFrame shape: (1068473, 23)

New DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068473 entries, 0 to 1068472
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   TaggedPitchType   1068473 non-null  object 
 1   AutoPitchType     1053452 non-null  object 
 2   PitchCall         1068473 non-null  object 
 3   KorBB             1068473 non-null  object 
 4   TaggedHitType     1068473 non-null  object 
 5   PlayResult        1068473 non-null  object 
 6   RunsScored        1068473 non-null  int64  
 7   RelSpeed          1060584 non-null  float64
 8   VertRelAngle      1060584 non-null  float64
 9   HorzRelAngle      1060584 non-null  float64
 10  SpinRate          1060067 non-null  float64
 11  SpinAxis          1053452 non-null  float64
 12  Tilt              1053452 non-null  object 
 13  RelHeight         1060584 non-null  float64
 14  R

In [38]:
# Look at unique values in PlayResult and PitchCall columns
print("\nUnique values in PlayResult:")
print(df_relevant['PlayResult'].unique())

print("\nUnique values in PitchCall:")
print(df_relevant['PitchCall'].unique())



Unique values in PlayResult:
['Undefined' 'Single' 'Out' 'HomeRun' 'Double' 'Error' 'Sacrifice'
 'FieldersChoice' 'Triple' 'StolenBase' 'CaughtStealing' 'error']

Unique values in PitchCall:
['BallCalled' 'StrikeCalled' 'FoulBallNotFieldable' 'StrikeSwinging'
 'InPlay' 'HitByPitch' 'FoulBallFieldable' 'FoulBall' 'BallinDirt'
 'Undefined' 'BallIntentional' 'StrkeSwinging ']


In [39]:
import numpy as np
def calculate_pitch_result(row):
    if pd.notna(row['PlayResult']):
        if row['PlayResult'] == 'Single':
            return 'Single'
        elif row['PlayResult'] == 'Double':
            return 'Double'
        elif row['PlayResult'] == 'Triple':
            return 'Triple'
        elif row['PlayResult'] == 'HomeRun':
            return 'HomeRun'
        elif row['PlayResult'] == 'Undefined':  # Fixed capitalization
            if pd.notna(row['PitchCall']):
                if row['PitchCall'] == 'BallCalled':
                    return 'Ball'
                elif row['PitchCall'] == 'FoulBallNotFieldable':
                    return 'FoulBall'
                elif row['PitchCall'] == 'FoulBallFieldable':
                    return 'FoulBall'
                elif row['PitchCall'] == 'FoulBall':
                    return 'FoulBall'
                elif row['PitchCall'] == 'StrikeSwinging':
                    return 'Strike'
                elif row['PitchCall'] == 'StrikeCalled':
                    return 'Strike'
    return np.nan  # Return np.nan for any unaccounted combinations

# Apply the updated function
df_relevant['PitchResult'] = df_relevant.apply(calculate_pitch_result, axis=1)


/var/folders/4r/yb6kq92n7nv16sy9y4wfcjww0000gp/T/ipykernel_33676/4039161935.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant['PitchResult'] = df_relevant.apply(calculate_pitch_result, axis=1)


In [40]:
# Remove rows where PitchResult is NaN
df_relevant_clean = df_relevant.dropna(subset=['PitchResult'])

# Print shape before and after to see how many rows were removed
print(f"Original shape: {df_relevant.shape}")
print(f"Shape after removing NaN PitchResult: {df_relevant_clean.shape}")


Original shape: (1068473, 24)
Shape after removing NaN PitchResult: (922496, 24)


In [41]:
# Get lists of numerical and non-numerical columns
numerical_columns = df_relevant_clean.select_dtypes(include=['int64', 'float64']).columns
non_numerical_columns = df_relevant_clean.select_dtypes(exclude=['int64', 'float64']).columns

print("\nNumerical columns:")
print(numerical_columns.tolist())

print("\nNon-numerical columns:")
print(non_numerical_columns.tolist())



Numerical columns:
['RunsScored', 'RelSpeed', 'VertRelAngle', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'InducedVertBreak', 'HorzBreak', 'VertApprAngle', 'HorzApprAngle', 'PlateLocHeight', 'PlateLocSide', 'ExitSpeed']

Non-numerical columns:
['TaggedPitchType', 'AutoPitchType', 'PitchCall', 'KorBB', 'TaggedHitType', 'PlayResult', 'Tilt', 'PitchResult']


In [42]:
# Calculate and display number of NaN values in numerical columns
nan_counts = df_relevant_clean[numerical_columns].isna().sum()
print("\nNumber of NaN values in numerical columns:")
print(nan_counts[nan_counts > 0])  # Only show columns that have NaN values

# Create a new dataset dropping NaN values except for ExitSpeed
columns_to_check = [col for col in numerical_columns if col != 'ExitSpeed']
df_relevant_no_nans = df_relevant_clean.dropna(subset=columns_to_check)

# Print shape before and after to see how many rows were removed
print(f"\nOriginal shape: {df_relevant_clean.shape}")
print(f"Shape after removing NaN values (except ExitSpeed): {df_relevant_no_nans.shape}")




Number of NaN values in numerical columns:
RelSpeed              5899
VertRelAngle          5899
HorzRelAngle          5899
SpinRate              6391
SpinAxis             11946
RelHeight             5899
RelSide               5899
Extension             6189
InducedVertBreak     11946
HorzBreak            11946
VertApprAngle         5899
HorzApprAngle         5899
PlateLocHeight        5899
PlateLocSide          5899
ExitSpeed           756338
dtype: int64

Original shape: (922496, 24)
Shape after removing NaN values (except ExitSpeed): (909839, 24)


In [20]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get numerical columns except ExitSpeed
numerical_features = df_relevant_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features.remove('ExitSpeed')
numerical_features.remove('RunsScored')

# Prepare features (X) and target (y)
X = df_relevant_clean[numerical_features]
y = df_relevant_clean['PitchResult']  # Using PitchCall as the target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [21]:
# Initialize and train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Print model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print feature importance
feature_importance = pd.DataFrame({
    'feature': numerical_features,
    'importance': rf_model.feature_importances_
})
print("\nFeature Importance:")
print(feature_importance.sort_values('importance', ascending=False))

Classification Report:
              precision    recall  f1-score   support

        Ball       0.93      0.97      0.95     79495
      Double       0.98      0.72      0.83      2558
    FoulBall       0.90      0.80      0.85     32625
     HomeRun       1.00      0.73      0.85      1526
      Single       0.99      0.71      0.83      8553
      Strike       0.85      0.90      0.88     59481
      Triple       0.98      0.71      0.82       262

    accuracy                           0.90    184500
   macro avg       0.95      0.79      0.86    184500
weighted avg       0.90      0.90      0.90    184500


Feature Importance:
             feature  importance
13      PlateLocSide    0.173021
12    PlateLocHeight    0.160260
10     VertApprAngle    0.068707
1       VertRelAngle    0.065082
2       HorzRelAngle    0.057597
11     HorzApprAngle    0.056124
6            RelSide    0.054999
5          RelHeight    0.054677
0           RelSpeed    0.052815
8   InducedVertBreak    0.052

In [43]:
# Get the depths of all trees in the Random Forest
tree_depths = [estimator.tree_.max_depth for estimator in rf_model.estimators_]

# Print the maximum, minimum, and average depth of the trees
print(f"Maximum tree depth: {max(tree_depths)}")
print(f"Minimum tree depth: {min(tree_depths)}")
print(f"Average tree depth: {np.mean(tree_depths):.2f}")


Maximum tree depth: 70
Minimum tree depth: 52
Average tree depth: 60.02


In [44]:
from sklearn.metrics import accuracy_score

# Training accuracy
y_train_pred = rf_model.predict(X_train)
training_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {training_accuracy:.2f}")

# Test accuracy (you already computed this indirectly, but re-confirming here)
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")



Training Accuracy: 1.00
Test Accuracy: 0.90


In [50]:
# Reinitialize the Random Forest model with a limited tree depth
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=27,  # Start with 15 as a reasonable value
    random_state=42,
    n_jobs=-1
)

# Retrain the model
rf_model.fit(X_train, y_train)

# Recompute training and test accuracies
y_train_pred = rf_model.predict(X_train)
training_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy (with limited depth): {training_accuracy:.2f}")

y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy (with limited depth): {test_accuracy:.2f}")


Training Accuracy (with limited depth): 0.95
Test Accuracy (with limited depth): 0.85


# Model Performance

| Model Configuration       | Training Accuracy | Test Accuracy |
|---------------------------|-------------------|---------------|
| Original (max_depth=None: 50 to 70) | **100%**          | **90%**       |
| Limited Depth (max_depth=15) | **67%**          | **65%**       |
| Limited Depth (max_depth=20) | **79%**          | **72%**       |
| Limited Depth (max_depth=25) | **92%**          | **82%**       |
| Limited Depth (max_depth=27) | **95%**          | **85%**       |
| Limited Depth (max_depth=30) | **98%**          | **88%**       |
